In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete = school_data_complete.sort_values(by="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
# total number of schools
total_schools = school_data['school_name'].count()

# total number of students
total_students = student_data['student_name'].count()
# student total will need to be formatted - str can't have calculations performed on them
total_students_formatted = "{:,}".format(total_students)

# total budget - help from kite.com
total_budget = "${:,.2f}".format(school_data['budget'].sum())

# average math score
average_math_score = round(school_data_complete['math_score'].mean(), 6)

# average reading score
average_reading_score = round(school_data_complete['reading_score'].mean(), 6)

# percentage of students with passing math score (70+)
plus70_math = 0
for score in school_data_complete['math_score']:
    if score >= 70:
        plus70_math = plus70_math + 1

percent_math_passing = round(((plus70_math / total_students) * 100), 6)

# percentage of students with passing reading score (70+)
plus70_reading = 0
for score in school_data_complete['reading_score']:
    if score >= 70:
        plus70_reading = plus70_reading + 1

percent_reading_passing = round(((plus70_reading / total_students) * 100), 6)

# percentage of students who passed math *and* reading (% Overall Passing)
plus70_overall = 0
for student in range(len(school_data_complete)):
    if school_data_complete['math_score'][student] >= 70 and school_data_complete['reading_score'][student] >= 70:
        plus70_overall = plus70_overall + 1

percent_overall_passing = round(((plus70_overall / total_students) * 100), 6)
print(percent_overall_passing)

65.172326


In [3]:
# dataframe for district summary
# help from Stack Overflow
district_summary = pd.DataFrame({
    "Total Schools" : total_schools,
    "Total Students" : total_students_formatted,
    "Total Budget" : total_budget,
    "Average Math Score" : average_math_score,
    "Average Reading Score" : average_reading_score,
    "% Passing Math" : percent_math_passing,
    "% Passing Reading" : percent_reading_passing,
    "% Passing Overall" : percent_overall_passing}, index=[0])

# print district summary
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [4]:
# initiate dataframe for school summary
# school name, school type, total students, total budget can be pulled from school_data df
school_summary = school_data.loc[:, ["school_name", "type", "size", "budget"]]

# rename columns
school_summary = school_summary.rename(columns={
    "school_name": "School Name", 
    "type": "School Type", 
    "size": "Total Students", 
    "budget": "Total School Budget"})

# reset index
school_summary = school_summary.set_index("School Name")

# sort schools alphabetically
school_summary = school_summary.sort_values(by="School Name")

# instantiate remaining columns of summary
# because we want 'Per Student Budget' to be formatted as currency, we need it to contain str objects - the rest can hold floats
school_summary['Per Student Budget'] = "0"
school_summary[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']] = 0
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,0,0,0,0,0,0
Cabrera High School,Charter,1858,1081356,0,0,0,0,0,0
Figueroa High School,District,2949,1884411,0,0,0,0,0,0
Ford High School,District,2739,1763916,0,0,0,0,0,0
Griffin High School,Charter,1468,917500,0,0,0,0,0,0


In [5]:
# grab budget and student totals for each school
perSchool_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
perSchool_counts = school_data_complete["school_name"].value_counts()
# calculate per capita spending
perCapita_budget = perSchool_budget / perSchool_counts
school_summary['Per Student Budget'] = perCapita_budget

# average math and reading scores
averageMath_score = school_data_complete.groupby(["school_name"]).mean()["math_score"]
averageReading_score = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

school_summary['Average Math Score'] = averageMath_score
school_summary['Average Reading Score'] = averageReading_score

# percent passing math, reading, and overall
# capture all students whose math/reading score >= 70 and sort them by school
# https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
plus70_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby(["school_name"])
plus70_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(["school_name"])
plus70_overall = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)].groupby(["school_name"])

# calculate percent passing
perMath_passing = (plus70_math.math_score.size() / school_summary["Total Students"]) * 100
perReading_passing = (plus70_reading.reading_score.size() / school_summary["Total Students"]) * 100
perOverall_passing = (plus70_overall.math_score.size() / school_summary["Total Students"]) * 100

# store calculations in appropriate columns
school_summary['% Passing Math'] = perMath_passing
school_summary['% Passing Reading'] = perReading_passing
school_summary['% Overall Passing'] = perOverall_passing

# Total School Budget and Per Student Budget as currency
# help from Stack Overflow (https://stackoverflow.com/questions/35019156/pandas-format-column-as-currency)
school_summary['Total School Budget'] = school_summary['Total School Budget'].apply(lambda x: "${:,.2f}".format(x))
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].apply(lambda y: "${:,.2f}".format(y))

# print school summary
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [6]:
# top performing schools - sort and display the top 5 schools by % overall passing
topPerforming_summary = school_summary.sort_values(by='% Overall Passing', ascending=False)
topPerforming_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [7]:
# bottom performing schools - sort and display the bottom 5 schools by % overall passing
bottomPerforming_summary = school_summary.sort_values(by='% Overall Passing')
bottomPerforming_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [ ]:
# average math scores by grade


In [ ]:
# average reading scores by grade

In [ ]:
# scores by school spending
# Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)

In [ ]:
# scores by school size
# Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)

In [ ]:
# scores by school type
# Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)